#### GLCM (Gray Level Co-occurrence Matrix) Features
GLCM is a statistical method of examining texture that considers the spatial relationship between pixels in an image. It computes the frequency of co-occurrence of pixel pairs at a specified distance and orientation. The following features are extracted from the GLCM matrix to quantify image texture:

1. Dissimilarity

    Description: Measures how different neighboring pixel values are.

    Interpretation: Higher values indicate greater contrast between neighboring pixels.

2. Correlation

    Description: Measures how correlated a pixel is to its neighbor over the whole image.

    Interpretation: Values closer to 1 suggest high correlation between pixels, and values near 0 suggest less correlation.

3. Energy

    Description: Measures the uniformity or the smoothness of the image. Also known as Angular Second Moment (ASM).

    Interpretation: Higher values indicate uniform, smooth texture. Lower values suggest more variance or complexity in the texture.

4. Contrast

    Description: Measures the intensity contrast between a pixel and its neighbor over the entire image.

    Interpretation: Higher contrast values indicate more noticeable differences between adjacent pixel intensities, often related to the sharpness of edges.

5. Homogeneity

    Description: Measures the closeness of the distribution of elements in the GLCM matrix to the diagonal (i.e., how similar neighboring pixels are).

    Interpretation: Higher homogeneity values indicate more uniformity, with similar pixel intensities being more frequent in the neighborhood.

##### Parameters

1. Distance: The pixel offset or number of pixels between the reference and the neighbor pixel.

    - Typical values: 1, 2, 3, etc.

    - Effect: Larger distances capture more global texture, while smaller ones focus on finer details.

2. Angle (or Direction): The direction in which the co-occurrence is calculated.

    - Typical values (in degrees or radians):

        - 0° (→): Horizontal

        - 45° (↗): Diagonal

        - 90° (↑): Vertical

        - 135° (↖): Other diagonal

    - Effect: Different angles capture different directional textures.

3. Levels (or Number of Gray Levels): Number of distinct gray levels in the image used to create the matrix.

    - Common values: 8, 16, 32, 64, 256 (or reduced to lower levels to reduce computation).
    - Effect: Reducing levels simplifies the matrix but may lose detail.

4. Symmetric or Asymmetric: Determines whether the GLCM is symmetric.

    - Symmetric: Counts (i, j) and (j, i) as the same.
    - Asymmetric: Counts (i, j) and (j, i) separately.
    - Effect: Symmetric matrices are often used to reduce complexity

5. Normalization: Whether the GLCM values are normalized to represent probabilities.

    - Normalized GLCM: Each element is divided by the sum of all elements.
    - Effect: Useful when comparing GLCMs across images of different sizes.

#### For more information follow the below links:

[Feature Extraction of Images using GLCM (Gray Level Cooccurrence Matrix)](https://medium.com/@girishajmera/feature-extraction-of-images-using-glcm-gray-level-cooccurrence-matrix-e4bda8729498)

[GLCM Texture Features](https://scikit-image.org/docs/stable/auto_examples/features_detection/plot_glcm.html)

[Gray-Level Co-Occurrence Matrix (GLCM): A Comprehensive Analysis](https://medium.com/aimonks/gray-level-co-occurrence-matrix-glcm-a-comprehensive-analysis-a58bbc49c0f6#:~:text=Texture%20Features%20from%20GLCM&text=These%20features%2C%20including%20contrast%2C%20correlation,the%20presence%20of%20sharp%20transitions.)

[exture Analysis Using the Gray-Level Co-Occurrence Matrix (GLCM)](https://ww2.mathworks.cn/help/images/texture-analysis-using-the-gray-level-co-occurrence-matrix-glcm.html)


In [10]:
import matplotlib.pyplot as plt
from skimage.feature import graycomatrix, graycoprops
import cv2 as cv
import numpy as np
import pandas as pd

# --- Step 1: Load and preprocess image ---
original_image = cv.imread('cancer.jpg')
gray_image = cv.cvtColor(original_image, cv.COLOR_BGR2GRAY)

# --- Step 2: Define parameters ---
angles = [0, np.pi / 4, np.pi / 2, np.pi * 3 / 4]
distances = [1, 2, 3, 4, 5]
Roi_size = 30

# Coordinates: (row, column) == (y, x)
cancer_locations = [(186, 198), (220, 198)]
normal_locations = [(155, 150), (315, 357)]

# --- Step 3: Extract ROIs ---
cancer_roi = [
    gray_image[y:y + Roi_size, x:x + Roi_size] for (y, x) in cancer_locations
]
normal_roi = [
    gray_image[y:y + Roi_size, x:x + Roi_size] for (y, x) in normal_locations
]

# --- Step 4: Initialize feature lists ---
angles_column = []
distances_column = []
dissimilarity = []
correlation = []
energy = []
contrast = []
homogeneity = []
labels = []

# --- Step 5: Extract GLCM features ---
for idx, patch in enumerate(cancer_roi + normal_roi):
    label = "cancer" if idx < len(cancer_roi) else "normal"
    for angle in angles:
        for dis in distances:
            glcm = graycomatrix(
                patch,
                distances=[dis],
                angles=[angle],
                levels=256,
                symmetric=True,
                normed=True
            )
            angles_column.append(int(np.degrees(angle)))
            distances_column.append(dis)
            dissimilarity.append(graycoprops(glcm, 'dissimilarity')[0, 0])
            correlation.append(graycoprops(glcm, 'correlation')[0, 0])
            energy.append(graycoprops(glcm, 'energy')[0, 0])
            contrast.append(graycoprops(glcm, 'contrast')[0, 0])
            homogeneity.append(graycoprops(glcm, 'homogeneity')[0, 0])
            labels.append(label)

# --- Step 6: Create DataFrame ---
features_df = pd.DataFrame({
    'angle': angles_column,
    'distance': distances_column,
    'dissimilarity': dissimilarity,
    'correlation': correlation,
    'energy': energy,
    'contrast': contrast,
    'homogeneity': homogeneity,
    'label': labels
})

# --- Step 7: Show or save the result ---
# print(features_df.head())
display(features_df)
# Optional: save to CSV
# features_df.to_csv("glcm_features.csv", index=False)


,angle,distance,dissimilarity,correlation,energy,contrast,homogeneity,label
0,0,1,1.434483,0.976695,0.078245,3.567816,0.472371,cancer
1,0,2,2.704762,0.922632,0.061956,11.792857,0.294233,cancer
2,0,3,3.692593,0.856606,0.056723,21.667901,0.223491,cancer
3,0,4,4.321795,0.799398,0.055477,29.996154,0.204579,cancer
4,0,5,4.608000,0.766099,0.054414,34.490667,0.197877,cancer
...,...,...,...,...,...,...,...,...
75,135,1,6.863258,0.920799,0.044315,104.130797,0.185610,normal
76,135,2,6.863258,0.920799,0.044315,104.130797,0.185610,normal
77,135,3,12.451531,0.716580,0.039397,353.584184,0.116933,normal
78,135,4,16.733882,0.462455,0.037692,647.578875,0.094613,normal
